In [ ]:
# make all imports 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [ ]:
# parse command line arguments 
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
    help="path to input dataset")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
    help="path to output loss/accuracy plot")
ap.add_argument("-m", "--model", type=str,
    default="mask_detector.model",
    help="path to output face mask detector model")
args = vars(ap.parse_args())

In [ ]:
# define deep learning paramaters 
INIT_LR = 1e-4 # inital learning rate
EPOCHS = 20 # how many cycles to train for 
BS = 32 #batch size

In [ ]:
# load and preprocess training data 
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"])) #grab all image paths in the dataset
data = [] #initalize the data and label lists 
labels = []

# loop over the image path and extract the class label 
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2] 
    
    #load the image and preprocess it 
    image = load_img(imagePath, target_size=(224, 224)) #resize images to 224x224 pixels 
    image = img_to_array(image) #convert to array format 
    image = preprocess_input(image) #scale pixel intensities to the range [-1,1]
    
    #update the data and label lists, convert them to numpy arrays 
    data.append(image) # append the preprocessed image and associated label to their respective lists 
    labels.append(label)
    data = np.array(data, dtype="float32") #ensure training data is in numpy array format 
    labels = np.array(labels)

In [ ]:
# prepare for data augmentation 

#one hot encode the labels -- used to convert categorical data to integer data
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

#partition the dataset into training and testing splits ( 80 - 20 )
trainX, testX, trainY, testY = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

#construct the training image generator for data augmentation be defining the data augmentation parameters
aug = ImageDataGenerator( 
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


In [ ]:
# Prepare the MobileVetV2 CNN architecture for fine tuning 

#load the MobileNet with pretrained ImageNet weights, leaving off the head
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

#construct a new fully connected head -- this will be placed on top of the base model 
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# append the newly constructed FC head to the base model 
model = Model(inputs=baseModel.input, outputs=headModel)

#loop over all layers in the base model and freeze them so they don't update during the first training cycle
for layer in baseModel.layers:
    layer.trainable = False
    
# At this point, the data is prepared and we have our model architecture in place 

In [ ]:
# Compile the model 
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# the model was compiled with the Adam optimizer, a learning rate decay schedule and binary cross entropy

In [ ]:
# Train the head of the network 
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS), # previously defined aug object provides batches of mutated image data
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

In [ ]:
#Evaluate the model on the test set 

#make predictions 
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

predIdxs = np.argmax(predIdxs, axis=1)

# print a classification report to the terminal for inspection 
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

#serialize the model to disk
print("[INFO] saving mask detector model...")
model.save(args["model"], save_format="h5")

In [ ]:
#Plot accuracy curves using matplotlib 
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"]) # Once ready, save the plot to disk